### Configuração inicial

In [42]:
import pathlib
import sys

requirements_filename = 'requirements_ap8.txt'
if str(pathlib.Path().resolve().name) == "notebooks":
    root = str(pathlib.Path().resolve().parent.parent)+'/'
    sys.path.append(root)
    requirements_path = root + 'requirements/'+requirements_filename
else:
    ! git clone --branch ap8 https://github.com/Fabio-Trindade/Eng-Aprendizado-Maquina.git
    root = str(pathlib.Path().resolve()) +'/'
    src_path = root + '/Eng-Aprendizado-Maquina/'
    requirements_path = src_path + 'requirements/'+requirements_filename
    sys.path.append(root + '/Eng-Aprendizado-Maquina/')

! pip install -r $requirements_path

### Imports

In [43]:
from src.utils.util_read_file import UtilReadFile
import pandas as pd
from sklearn.model_selection import train_test_split
from src.utils.util_pre_processor import UtilPreProcessor
from sklearn.preprocessing import MinMaxScaler

### Lendo *Dataset*

In [44]:
df = UtilReadFile.read_csv_with_pandas(root+'datasets/churn_bank.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


### Definindo Dados de Treinamento

In [45]:
feature_columns = ['CreditScore', 'Geography',
        'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
        'IsActiveMember', 'EstimatedSalary']
feature_columns

['CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [46]:
target_column = ['Exited']
target_column

['Exited']

### Separando os dados categóricos

In [47]:
categorical_columns = ['Geography', 'Gender']

### Separando os dados numéricos

In [48]:
numeric_columns = list(set(feature_columns) - set(categorical_columns))
numeric_columns

['Tenure',
 'Balance',
 'Age',
 'IsActiveMember',
 'CreditScore',
 'HasCrCard',
 'EstimatedSalary',
 'NumOfProducts']

## Engenharia de atributos - Dados Numéricos

### Normalização dos dados

In [50]:
scaler = MinMaxScaler()
columns_to_scale = ["CreditScore", "Age", "Balance",'EstimatedSalary']
scaler.fit(df[columns_to_scale])

df[columns_to_scale] = scaler.transform(df[columns_to_scale])
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,0.538,France,Female,0.324324,2,0.000000,1,1,1,0.506735,1
1,2,15647311,Hill,0.516,Spain,Female,0.310811,1,0.334031,1,0,1,0.562709,0
2,3,15619304,Onio,0.304,France,Female,0.324324,8,0.636357,3,1,0,0.569654,1
3,4,15701354,Boni,0.698,France,Female,0.283784,1,0.000000,2,0,0,0.469120,0
4,5,15737888,Mitchell,1.000,Spain,Female,0.337838,2,0.500246,1,1,1,0.395400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,0.842,France,Male,0.283784,5,0.000000,2,1,0,0.481341,0
9996,9997,15569892,Johnstone,0.332,France,Male,0.229730,10,0.228657,1,1,1,0.508490,0
9997,9998,15584532,Liu,0.718,France,Female,0.243243,7,0.000000,1,0,1,0.210390,1
9998,9999,15682355,Sabbatini,0.844,Germany,Male,0.324324,3,0.299226,2,1,0,0.464429,1


#### Binning da idade

In [51]:
bins = [0,26,56,76,100]

new_df = df[list((feature_columns)+(target_column))].copy()
df['Age'] = pd.cut(new_df['Age'], bins=bins, labels=False)
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,0.538,France,Female,0.0,2,0.000000,1,1,1,0.506735,1
1,2,15647311,Hill,0.516,Spain,Female,0.0,1,0.334031,1,0,1,0.562709,0
2,3,15619304,Onio,0.304,France,Female,0.0,8,0.636357,3,1,0,0.569654,1
3,4,15701354,Boni,0.698,France,Female,0.0,1,0.000000,2,0,0,0.469120,0
4,5,15737888,Mitchell,1.000,Spain,Female,0.0,2,0.500246,1,1,1,0.395400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,0.842,France,Male,0.0,5,0.000000,2,1,0,0.481341,0
9996,9997,15569892,Johnstone,0.332,France,Male,0.0,10,0.228657,1,1,1,0.508490,0
9997,9998,15584532,Liu,0.718,France,Female,0.0,7,0.000000,1,0,1,0.210390,1
9998,9999,15682355,Sabbatini,0.844,Germany,Male,0.0,3,0.299226,2,1,0,0.464429,1


### Engenharia de atributos - Dados Categóricos

### Transformação dos dados categóricos

In [52]:
vocabs = {}
for column in categorical_columns:
    vocabs[column] = UtilPreProcessor.creat_vocab(df[column])[0]
    for i in range(len(df)):
        df.loc[i,column] = UtilPreProcessor.get_index_vocab(vocabs[column],df.loc[i,column])
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,0.538,0,0,0.0,2,0.000000,1,1,1,0.506735,1
1,2,15647311,Hill,0.516,1,0,0.0,1,0.334031,1,0,1,0.562709,0
2,3,15619304,Onio,0.304,0,0,0.0,8,0.636357,3,1,0,0.569654,1
3,4,15701354,Boni,0.698,0,0,0.0,1,0.000000,2,0,0,0.469120,0
4,5,15737888,Mitchell,1.000,1,0,0.0,2,0.500246,1,1,1,0.395400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,0.842,0,1,0.0,5,0.000000,2,1,0,0.481341,0
9996,9997,15569892,Johnstone,0.332,0,1,0.0,10,0.228657,1,1,1,0.508490,0
9997,9998,15584532,Liu,0.718,0,0,0.0,7,0.000000,1,0,1,0.210390,1
9998,9999,15682355,Sabbatini,0.844,2,1,0.0,3,0.299226,2,1,0,0.464429,1


### Treinamento do Modelo com Validação Cruzada

#### Separação dos Dados em Treino e Teste

In [53]:
df_train, df_test = train_test_split(df[feature_columns + target_column], test_size=0.2, random_state=42)
df_train = df_train.reset_index().drop('index',axis=1)
df_test = df_test.reset_index().drop('index',axis=1)

df_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.672,0,1,0.0,6,0.000000,2,1,1,0.895494,0
1,0.564,2,1,0.0,4,0.476786,2,1,1,0.979930,0
2,0.418,1,1,0.0,3,0.457317,1,1,0,0.429438,1
3,0.422,0,0,0.0,9,0.540606,1,1,0,0.765417,1
4,0.334,0,1,0.0,9,0.566554,1,0,0,0.197401,1
...,...,...,...,...,...,...,...,...,...,...,...
7995,0.836,0,1,0.0,8,0.277853,1,1,1,0.346880,0
7996,0.664,0,0,0.0,1,0.000000,1,1,1,0.003475,0
7997,0.770,0,0,0.0,1,0.000000,3,0,0,0.461087,1
7998,0.634,0,1,0.0,8,0.758186,1,1,0,0.487529,1


In [54]:
import autokeras as ak
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=42)
auto_models = []
for train_index, test_index in kf.split(df_train):
    train_df = df_train.iloc[train_index]
    val_df = df_train.iloc[test_index]  
    automodel = ak.AutoModel(
        ak.Input(),
        ak.ClassificationHead(),
    )
    automodel.fit(train_df[feature_columns].values.astype(float),
                  train_df[target_column].values.astype(float), 
                  epochs=10,
                  validation_data = (val_df[feature_columns].values.astype(float),
                                      val_df[target_column].values.astype(float)))
    auto_models.append(automodel)

Reloading Tuner from ./auto_model/tuner0.json


2024-04-30 12:12:20.017867: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7893 - loss: 0.5485 - val_accuracy: 0.8087 - val_loss: 0.4912
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7918 - loss: 0.5134 - val_accuracy: 0.8087 - val_loss: 0.4906
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7918 - loss: 0.5139 - val_accuracy: 0.8087 - val_loss: 0.4901
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7918 - loss: 0.5136 - val_accuracy: 0.8087 - val_loss: 0.4894
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7918 - loss: 0.5143 - val_accuracy: 0.8087 - val_loss: 0.4895
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7918 - loss: 0.5140 - val_accuracy: 0.8087 - val_loss: 0.4897
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7918 - loss: 0.5134 - val_accuracy: 0.8087 - val_loss: 0.4895
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7918 - loss: 0.5148 - val_accuracy: 0

2024-04-30 12:12:41.717264: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7923 - loss: 0.5568 - val_accuracy: 0.7825 - val_loss: 0.5251
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7948 - loss: 0.5091 - val_accuracy: 0.7825 - val_loss: 0.5255
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7948 - loss: 0.5095 - val_accuracy: 0.7825 - val_loss: 0.5254
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7948 - loss: 0.5105 - val_accuracy: 0.7825 - val_loss: 0.5252
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7948 - loss: 0.5088 - val_accuracy: 0.7825 - val_loss: 0.5254
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7948 - loss: 0.5089 - val_accuracy: 0.7825 - val_loss: 0.5260
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7948 - loss: 0.5097 - val_accuracy: 0.7825 - val_loss: 0.5250
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7948 - loss: 0.5098 - val_accuracy: 0.

...

Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7870 - loss: 0.5569 - val_accuracy: 0.7900 - val_loss: 0.5140
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7936 - loss: 0.5110 - val_accuracy: 0.7900 - val_loss: 0.5140
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7936 - loss: 0.5119 - val_accuracy: 0.7900 - val_loss: 0.5141
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7936 - loss: 0.5121 - val_accuracy: 0.7900 - val_loss: 0.5141
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7936 - loss: 0.5113 - val_accuracy: 0.7900 - val_loss: 0.5141
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7936 - loss: 0.5113 - val_accuracy: 0.7900 - val_loss: 0.5143
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7936 - loss: 0.5103 - val_accuracy: 0.7900 - val_loss: 0.5142
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7936 - loss: 0.5111 - val_accuracy: 0.

2024-04-30 12:14:51.581088: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7914 - loss: 0.5584 - val_accuracy: 0.8075 - val_loss: 0.4908
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7922 - loss: 0.5143 - val_accuracy: 0.8075 - val_loss: 0.4910
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7922 - loss: 0.5127 - val_accuracy: 0.8075 - val_loss: 0.4906
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7922 - loss: 0.5140 - val_accuracy: 0.8075 - val_loss: 0.4908
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7922 - loss: 0.5144 - val_accuracy: 0.8075 - val_loss: 0.4908
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7922 - loss: 0.5135 - val_accuracy: 0.8075 - val_loss: 0.4908
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7922 - loss: 0.5128 - val_accuracy: 0.8075 - val_loss: 0.4903
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7922 - loss: 0.5126 - val_accuracy: 0.

2024-04-30 12:15:11.829908: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7862 - loss: 0.5578 - val_accuracy: 0.8138 - val_loss: 0.4828
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7921 - loss: 0.5141 - val_accuracy: 0.8138 - val_loss: 0.4827
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7921 - loss: 0.5149 - val_accuracy: 0.8138 - val_loss: 0.4825
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7921 - loss: 0.5141 - val_accuracy: 0.8138 - val_loss: 0.4827
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7921 - loss: 0.5132 - val_accuracy: 0.8138 - val_loss: 0.4820
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7921 - loss: 0.5133 - val_accuracy: 0.8138 - val_loss: 0.4818
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7921 - loss: 0.5145 - val_accuracy: 0.8138 - val_loss: 0.4819
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7921 - loss: 0.5145 - val_accuracy: 0.

### Avaliação do Modelo

In [55]:
for automodel in auto_models:
    print(automodel.evaluate(df_test[feature_columns].values.astype(float), df_test[target_column].values.astype(float)))

 1/63 ━━━━━━━━━━━━━━━━━━━━ 9s 151ms/step - accuracy: 0.8125 - loss: 0.6658

/home/fabio/venvs/my_venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8134 - loss: 0.6820  
[0.6802976131439209, 0.815500020980835]
 1/63 ━━━━━━━━━━━━━━━━━━━━ 9s 158ms/step - accuracy: 0.8125 - loss: 0.6658

/home/fabio/venvs/my_venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8134 - loss: 0.6820  
[0.6802976131439209, 0.815500020980835]
 1/63 ━━━━━━━━━━━━━━━━━━━━ 9s 150ms/step - accuracy: 0.8125 - loss: 0.6658

...

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.8134 - loss: 0.6820
[0.6802976131439209, 0.815500020980835]
57/63 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step - accuracy: 0.8130 - loss: 0.6826

/home/fabio/venvs/my_venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step - accuracy: 0.8134 - loss: 0.6820
[0.6802976131439209, 0.815500020980835]
58/63 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - accuracy: 0.8131 - loss: 0.6825

/home/fabio/venvs/my_venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.8134 - loss: 0.6820
[0.6802976131439209, 0.815500020980835]


O melhor resultado obtido na atividade 3 foi de 81% de acurácia, enquanto utilizando Auto-ML com *autokeras* foi de 81,3%. Neste *dataset*, ao utilizar a mesma etapa de pré-processamento da atividade 3, a ferramenta de Auto-ML *autokeras* não conseguiu melhorar siginficativamente os resultados.